In [1]:
import pandas as pd
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
from utils import *

In [2]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]


In [3]:

# Create a VideoCapture object and read from the input file
cap = cv2.VideoCapture('./videos/stool.avi')

# Check if the camera opened successfully
if not cap.isOpened():
  print("Error opening video file")
  exit()

# Read until video is completed
while cap.isOpened():
  # Capture frame-by-frame
  ret, frame = cap.read()

  # If the frame is not read correctly, break
  if not ret:
    break

  # Display the resulting frame
  cv2.imshow("Video", frame)

  # Press Q on keyboard to exit
  cv2.waitKey(33)
    

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()


In [5]:
STANDING = 0
RIGHT_LIFT = 1
LEFT_LIFT = 2
def detect_step(action,action_array,count_dict):
    print(action_array)
    if len(action_array) == 0 and action != STANDING:
        return
    if len(action_array) == 0 or action_array[-1] != action:
        action_array.append(action)
    if len(action_array) == 3:
        print(action_array)
        first_element,second_element,third_element = action_array
        if first_element == STANDING and second_element == RIGHT_LIFT and third_element == STANDING:
            count_dict['right'] += 1
        if first_element == STANDING and second_element == LEFT_LIFT and third_element == STANDING:
            count_dict['left'] += 1
        action_array.clear()
    return

def stepping_on_stool(req_angles,count_dict,action_array):
    left_stand_flag = False
    right_stand_flag = False

    left_lift_flag = False
    right_lift_flag = False

    standing_position = 180
    lifted_postition = 135
    left_knee_angle = req_angles[6]
    right_knee_angle = req_angles[7]
    if standing_position - epsilon_knee <= left_knee_angle <= standing_position + epsilon_knee:
        # print('left leg in standing position')
        left_stand_flag = True
        left_lift_flag = False
    if standing_position - epsilon_knee <= right_knee_angle <= standing_position + epsilon_knee:
        right_stand_flag = True
        right_lift_flag = False
        # print('right leg in standing position')

    if lifted_postition - epsilon_knee <= left_knee_angle <= lifted_postition + epsilon_knee:
        # print('left leg in standing position')
        left_lift_flag = True
        left_stand_flag = False
    if lifted_postition - epsilon_knee <= right_knee_angle <= lifted_postition + epsilon_knee:
        right_lift_flag = True
        right_stand_flag = False
        # print('right leg in standing position')
    if left_stand_flag and right_stand_flag:
        print("Person is standing.")
        detect_step(STANDING,action_array,count_dict)
    if left_stand_flag and right_lift_flag:
        print("Person has placed right foot on stool.")
        detect_step(RIGHT_LIFT,action_array,count_dict)
    if right_stand_flag and left_lift_flag:
        print('Person has placed left foot on stool.')
        detect_step(LEFT_LIFT,action_array,count_dict)

    return count_dict

In [8]:
###### ALGORITHM HYPER PARAMETERS########
epsilon_knee = 10
epsilon_rest_of_the_body = 5
###FLAGS###
start_flag = False
count_dict = dict({'left':0,'right':0})
action_array = []
##########################################
cap = cv2.VideoCapture('./videos/stool.avi')
frame_rate = cap.get(cv2.CAP_PROP_FPS)
font = cv2.FONT_HERSHEY_SIMPLEX
# text = "Hello, OpenCV!"
frame_idx = 0
start_time = cv2.getTickCount()
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    frame_idx+=1
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
      ######################################################
      #Extract Joint coordinates
      my_landmarks = results.pose_landmarks.landmark
      joint_coordinates = get_coordinates(my_landmarks)
      #calculate joint angles
      req_angles = get_all_angles(joint_coordinates)
      req_angles = np.around(req_angles,2)
      #display joint angles
      for i in range(len(req_angles)):
          cv2.putText(
              image,
              str(req_angles[i]),
              tuple(np.multiply(joint_coordinates[i][1],[640, 480]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
          )
      left_knee_angle = req_angles[6]
      right_knee_angle = req_angles[7]
      key = cv2.waitKey(33) & 0xFF
      if key == ord('s'):
         print('Exercise Started')
         start_flag = True
      if start_flag:
         res = stepping_on_stool(req_angles,count_dict,action_array)
         if res:
          left_count = res['left']
          left_text = f'LEFT STEPS: {left_count}'
          right_count = res['right']
          right_text = f'RIGHT STEPS: {right_count}'
          overall_count_as_str = str(left_count + right_count)
          cv2.rectangle(image, (0, 0), (640,60), (255, 0, 0), -1)
          cv2.putText(image,left_text, (50, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
          cv2.putText(image,right_text, (300, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
      #######################################################
          end_time = cv2.getTickCount()   
          fps = (end_time - start_time) / cv2.getTickFrequency()
          # print("the FPS is :",fps)
          adjusted_fps = 30/(frame_idx/fps)
          print("elapsed time is: ", fps/(adjusted_fps))
          # print("The fps is :", frame_idx/fps) 
    except Exception as e:
        print('error',e)
        pass

    # Flip the image horizontally for a selfie-view display.
    # flipped_frame = cv2.flip(image, 0)
    cv2.imshow('MediaPipe Holistic',image)
    if cv2.waitKey(33) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

Exercise Started
Person is standing.
[]
elapsed time is:  0.6666666666666666
Person is standing.
[0]
elapsed time is:  0.7000000000000001
Person is standing.
[0]
elapsed time is:  0.7333333333333333
Person is standing.
[0]
elapsed time is:  0.7666666666666667
Person is standing.
[0]
elapsed time is:  0.7999999999999999
Person is standing.
[0]
elapsed time is:  0.8333333333333333
Person is standing.
[0]
elapsed time is:  0.8666666666666667
Person is standing.
[0]
elapsed time is:  0.9
Person is standing.
[0]
elapsed time is:  0.9333333333333335
Person is standing.
[0]
elapsed time is:  0.9666666666666666
Person is standing.
[0]
elapsed time is:  1.0
Person is standing.
[0]
elapsed time is:  1.0333333333333334
Person is standing.
[0]
elapsed time is:  1.0666666666666667
elapsed time is:  1.1
elapsed time is:  1.1333333333333333
elapsed time is:  1.1666666666666667
elapsed time is:  1.2
elapsed time is:  1.2333333333333332
elapsed time is:  1.2666666666666666
elapsed time is:  1.3
elapsed